In [1]:
import faiss
import numpy as np 
from sentence_transformers import SentenceTransformer
import re

/home/akayo/Рабочий стол/PetProjects/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def chunk_seperator(doc: str, max_len=80):
    # Удаляем нежелательные символы (#, *)
    doc = re.sub(r'[\#\*]', '', doc)
    
    # Разбиваем документ на слова
    words = doc.split()
    
    chunks = []
    current_chunk = []
    word_count = 0
    
    # Проходим по каждому слову
    for word in words:
        # Если текущее количество слов меньше max_len, добавляем слово в чанк
        if word_count < max_len:
            current_chunk.append(word)
            word_count += 1
        else:
            # Если достигли максимума, сохраняем текущий чанк и начинаем новый
            chunks.append(' '.join(current_chunk))
            current_chunk = [word]  # Начинаем новый чанк с текущего слова
            word_count = 1  # Сбрасываем счетчик слов
    
    # Добавляем последний чанк, если он не пустой
    if current_chunk:
        chunks.append(' '.join(current_chunk))
    
    return chunks

In [3]:
with open('youtube.txt', 'r', encoding='utf-8') as file:
    youtube_data = file.read()

data = chunk_seperator(youtube_data, max_len=40)

In [4]:
model = SentenceTransformer('intfloat/e5-base')

doc_embeddings = model.encode(data, normalize_embeddings=True)


In [5]:
dimension = doc_embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)
index.add(doc_embeddings)

In [6]:
query = 'Основые функции Youtube'
query_embedding = model.encode(query, normalize_embeddings=True)
query_embedding = np.expand_dims(query_embedding, axis=0)

In [7]:
k = 2
distances, indices = index.search(query_embedding, k)

In [8]:
for i, idx in enumerate(indices[0]):
    print(f"Релевантный документ {i+1}: {data[idx]} (дистанция: {distances[0][i]})")

Релевантный документ 1: знаний о YouTube. Вы можете использовать её для изучения платформы или создания документации. Если нужно больше информации по конкретной теме, дайте знать! (дистанция: 0.8943448066711426)
Релевантный документ 2: Ах, я понял! Вы хотите текстовую базу знаний о YouTube — информацию о платформе, её функциях, терминах и особенностях. Вот подробная информация: --- База знаний о YouTube Что такое YouTube? YouTube — это крупнейшая в мире платформа для просмотра, загрузки (дистанция: 0.8880485892295837)
